In [2]:
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
from IPython.display import HTML

# -----------------------------
# Earth
# -----------------------------
earth_trace = go.Scatter3d(
    x=[0], y=[0], z=[0],
    mode="markers+text",
    marker=dict(size=36, color="blue"),
    text=["Earth"],
    textfont=dict(color="white", size=14),
    textposition="top center",
    name="Earth"
)

# -----------------------------
# NEO data (real diameters in km)
# -----------------------------
neos = [
    {"name": "Apophis",  "diameter": 0.34,   "color": "beige"},
    {"name": "Bennu",    "diameter": 0.482,  "color": "saddlebrown"},
    {"name": "Apollo",   "diameter": 1.5,    "color": "goldenrod"},
    {"name": "Toutatis", "diameter": 5.4,    "color": "lightgray"},
    {"name": "Eros",     "diameter": 16.84,  "color": "dimgray"},
    {"name": "Ganymed",  "diameter": 37.675, "color": "firebrick"}
]

neo_traces = []
orbit_traces = []
neo_radii = []
neo_speeds = []

# -----------------------------
# Orbit + marker setup
# -----------------------------
for i, neo in enumerate(neos):
    r = 2.0 + i * 1.0   # orbit radius (arbitrary scale units)
    neo_radii.append(r)

    # Kepler's 3rd law: speed ∝ 1 / r^(3/2)
    speed = 7.5 / (r ** 1.5)
    neo_speeds.append(speed)

    # Scale marker size from real diameter
    size_marker = np.log10(neo["diameter"]*100 + 10) * 3

    # Orbit path
    orbit_traces.append(
        go.Scatter3d(
            x = r*np.cos(np.linspace(0, 2*np.pi, 200)),
            y = r*np.sin(np.linspace(0, 2*np.pi, 200)),
            z = np.zeros(200),
            mode = "lines",
            line = dict(color="white", width=1),
            showlegend=False
        )
    )

    # NEO marker
    neo_traces.append(
        go.Scatter3d(
            x=[r], y=[0], z=[0],
            mode="markers+text",
            marker=dict(size=size_marker, color=neo["color"]),
            text=[neo["name"]],
            textfont=dict(color=neo["color"], size=12),
            textposition="top center",
            name=neo["name"]
        )
    )

# -----------------------------
# Layout
# -----------------------------
layout = go.Layout(
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        bgcolor="black",
        camera=dict(eye=dict(x=1.2, y=1.2, z=0.8))
    ),
    margin=dict(l=0, r=0, t=40, b=0),
    title="3D Simulation Of NEOs Orbiting Earth",
    showlegend=True
)

# -----------------------------
# Combine figure
# -----------------------------
fig = go.Figure(data=[earth_trace] + orbit_traces + neo_traces, layout=layout)

# -----------------------------
# JS animation loop (Kepler speed)
# -----------------------------
custom_js = r"""
<script>
var gd = document.querySelectorAll('.js-plotly-plot')[0];
function step() {
    var t = Date.now() / 500;
    var radii = """ + str(neo_radii) + """;
    var speeds = """ + str(neo_speeds) + """;

    var update_x = [];
    var update_y = [];
    var update_z = [];

    for (var i=0; i<radii.length; i++) {
        var angle = t * speeds[i];
        update_x.push([radii[i]*Math.cos(angle)]);
        update_y.push([radii[i]*Math.sin(angle)]);
        update_z.push([0]);
    }

    // offset = 1 (Earth) + len(orbit_traces)
    var offset = 1 + radii.length;
    var indices = [];
    for (var j=0; j<radii.length; j++) {
        indices.push(offset + j);
    }

    Plotly.restyle(gd, {x: update_x, y: update_y, z: update_z}, indices);
    requestAnimationFrame(step);
}
step();
</script>
"""

# Export HTML (Colab / Jupyter)
html_str = plot(fig, include_plotlyjs=True, output_type="div")
HTML(html_str + custom_js)


Output hidden; open in https://colab.research.google.com to view.